In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd


driver = webdriver.Chrome("/Users/ming/Desktop/goodreads/chromedriver")
driver.get("https://www.goodreads.com/")

# Login
driver.find_element_by_xpath(
    '//*[@id="userSignInFormEmail"]').send_keys('') # enter your email 
driver.find_element_by_xpath(
    '//*[@id="user_password"]').send_keys('') # enter your password
driver.find_element_by_xpath(
    '//*[@id="sign_in"]/div[3]/input[1]').click()


# Initialize variables
link = []
review = []
rating = []


# Specify page range (1 to 5)
for i in range(1, 6):
    string = "https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once?page=" + \
        str(i)
    driver.get(string)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = soup.findAll('a', {'class': 'bookTitle'}, {'itemprop': 'url'})
    # append book links
    for i in links:
        link.append(i['href'])


# Remove duplicates
link = list(set(link))
print("No. of books: ", len(link))


# Loop through the 300 books to scrape reviews
for i in range(len(link)):
    print("book num:", i)
    print("review/rating:", len(review), len(rating))
    driver.get(f"https://www.goodreads.com{link[i]}")
    soup2 = BeautifulSoup(driver.page_source, 'html.parser')

    # Scrape all 10 pages
    for x in range(10):

        soup2 = BeautifulSoup(driver.page_source, 'html.parser')
        for i in soup2.findAll(class_='friendReviews elementListBrown'):
            try:
                X = i.find(class_='readable').findAll('span')
                B = i.find(class_='staticStars notranslate').find('span')

                if (len(X) > 1):
                    C = i.find(class_='readable').findAll('span')[1]
                else:
                    C = i.find(class_='readable').find('span')

                if (v is not None for v in [B, C]):
                    rating.append(B.text)
                    review.append(C.text)
            except:
                pass
        time.sleep(0.5)
        try:
            wait = WebDriverWait(driver, 10)
            element = wait.until(EC.element_to_be_clickable(
                (By.CLASS_NAME, "next_page")))
            element.click()
            time.sleep(3.0)
        except Exception as e:
            print(e)
            pass

# Print summary
print("Summary")
print("No. of Reviews/Ratings:", len(review), len(rating))

# Convert to csv
df = pd.DataFrame()
df["review"] = review
df["rating"] = rating
df = df.drop_duplicates()

print("Final length:", len(df))
df.to_csv('reviews12_14.csv')
